## Project 6 - Hospitality App New User Bookings

In [1]:
## import necessary packages
import numpy as np
import pandas as pd
import time
import random
import datetime
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.style.use('ggplot')

In [2]:
train_data = pd.read_csv('data/train_users_2.csv')
test_data = pd.read_csv('data/test_users.csv')
age_gender = pd.read_csv('data/age_gender_bkts.csv')
countries = pd.read_csv('data/countries.csv')

In [3]:
print(train_data.shape)
print(train_data.columns)

(213451, 16)
Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination'],
      dtype='object')


In [4]:
print(test_data.shape)
print(test_data.columns)

(62096, 15)
Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser'],
      dtype='object')


In [5]:
np.setdiff1d(train_data.columns, test_data.columns)

array(['country_destination'], dtype=object)

## 1. Data Exploration

In [6]:
train_data.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [7]:
test_data.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 26.1+ MB


In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62096 entries, 0 to 62095
Data columns (total 15 columns):
id                         62096 non-null object
date_account_created       62096 non-null object
timestamp_first_active     62096 non-null int64
date_first_booking         0 non-null float64
gender                     62096 non-null object
age                        33220 non-null float64
signup_method              62096 non-null object
signup_flow                62096 non-null int64
language                   62096 non-null object
affiliate_channel          62096 non-null object
affiliate_provider         62096 non-null object
first_affiliate_tracked    62076 non-null object
signup_app                 62096 non-null object
first_device_type          62096 non-null object
first_browser              62096 non-null object
dtypes: float64(2), int64(2), object(11)
memory usage: 7.1+ MB


In [10]:
print ('training dataset ({} rows) null value:\n'.format(train_data.shape[0]))
print (train_data.isnull().sum())
print ('\n' + '***' * 10 + '\n')
print ('test dataset ({} rows) null value:\n'.format(test_data.shape[0]))
print (test_data.isnull().sum())

training dataset (213451 rows) null value:

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

******************************

test dataset (62096 rows) null value:

id                             0
date_account_created           0
timestamp_first_active         0
date_first_booking         62096
gender                         0
age                        28876
signup_method                  0
signup_flow                    0
language                       0
affiliate_channel             

## 2. Feature Engineering

In [11]:
train_row = train_data.shape[0]
# The label we need to predict
labels = train_data['country_destination'].values

# the id number is not useful for prediction. We need it for submission
id_test = test_data['id']

# drop the id column
# drop the date_first_booking since there is all NaN in test dataset which we figured out in the data exploration
# drop the label in training set
train_data.drop(['country_destination', 'date_first_booking'], axis = 1, inplace = True)
test_data.drop(['date_first_booking'], axis = 1, inplace = True)

In [12]:
full_data = pd.concat([train_data, test_data], axis = 0, ignore_index = True)

#### date_account_created （dac）

In [13]:
# create year, month, day feature for dac
dac = pd.to_datetime(full_data.date_account_created)
full_data['dac_year'] = np.array([x.year for x in dac])
full_data['dac_month'] = np.array([x.month for x in dac])
full_data['dac_day'] = np.array([x.day for x in dac])

In [14]:
# create features of weekday for dac, showing Monday to Sunday
full_data['dac_wd'] = np.array([x.isoweekday() for x in dac])
df_dac_wd = pd.get_dummies(full_data.dac_wd, prefix = 'dac_wd')
full_data = pd.concat((full_data, df_dac_wd), axis = 1)
full_data.drop(['dac_wd'], axis = 1, inplace = True)

In [15]:
def get_season(dt):
    dt = dt.date()
    # dt must be a datetime type
    if dt.month in [3,4,5]:
        return 'Spring'    
    elif dt.month in [6,7,8]:
        return 'Summer'    
    elif dt.month in [9,10,11]:
        return 'Fall'   
    else:
        return 'Winter' 

In [16]:
# create season features from dac
full_data['dac_season'] = np.array([get_season(x) for x in dac])
df_dac_season = pd.get_dummies(full_data.dac_season, prefix = 'dac_season')
full_data = pd.concat((full_data, df_dac_season), axis = 1)
full_data.drop(['dac_season'], axis = 1, inplace = True)

#### timestamp_first_active (tfa)

In [17]:
tfa = full_data.timestamp_first_active.astype(str).apply(lambda x: datetime.datetime(int(x[:4]),
                                                                          int(x[4:6]), 
                                                                          int(x[6:8]),
                                                                          int(x[8:10]),
                                                                          int(x[10:12]),
                                                                          int(x[12:])))

In [18]:
# create tfa_year, tfa_month, tfa_day feature
full_data['tfa_year'] = np.array([x.year for x in tfa])
full_data['tfa_month'] = np.array([x.month for x in tfa])
full_data['tfa_day'] = np.array([x.day for x in tfa])

In [19]:
# create features of weekday
full_data['tfa_wd'] = np.array([x.isoweekday() for x in tfa])
df_tfa_wd = pd.get_dummies(full_data.tfa_wd, prefix = 'tfa_wd')
full_data = pd.concat((full_data, df_tfa_wd), axis = 1)
full_data.drop(['tfa_wd'], axis = 1, inplace = True)

In [20]:
# create season features from tfa
full_data['tfa_season'] = np.array([get_season(x) for x in tfa])
df_tfa_season = pd.get_dummies(full_data.tfa_season, prefix = 'tfa_season')
full_data = pd.concat((full_data, df_tfa_season), axis = 1)
full_data.drop(['tfa_season'], axis = 1, inplace = True)

#### Time span between dac and tfa can also be used as a feature. 

In [21]:
dt_span = (dac - tfa).dt.seconds
full_data['dt_span'] = np.array([x for x in dt_span])

### Deal with Age

In [22]:
age = full_data['age']
age.fillna(-1, inplace = True)

In [23]:
## This are birthdays instead of age (estimating age by doing 2016 - value)
age = np.where(np.logical_and(age<2005, age>1900), 2020-age, age)
## This is the current year insted of age, we also consider this as N/A
age = np.where(np.logical_and(age<2016, age>2010), -1, age) 

In [24]:
## Keeping ages in 14 < age < 106 as OK
interval = 15

def get_age(age):
    # age is a float number   
    if age < 0:
        return 'NA'
    elif (age < interval):
        return interval
    elif (age <= interval * 2):
        return interval*2
    elif (age <= interval * 3):
        return interval * 3
    elif (age <= interval * 4):
        return interval * 4
    elif (age <= interval * 5):
        return interval * 5
    elif (age <= interval * 6):
        return interval * 6
    elif (age <= interval * 7):
        return interval * 7
    else:
        return 'Unphysical'

In [25]:
full_data['age'] = np.array([get_age(x) for x in age])
df_age = pd.get_dummies(full_data.age, prefix = 'age')

In [26]:
full_data = pd.concat((full_data, df_age), axis = 1)

## Label Encoding and One Hot Encoding

In [35]:
pd.set_option('display.max_columns', None) 
full_data

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day,dac_wd_1,dac_wd_2,dac_wd_3,dac_wd_4,dac_wd_5,dac_wd_6,dac_wd_7,dac_season_Fall,dac_season_Spring,dac_season_Summer,dac_season_Winter,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,tfa_season_Fall,tfa_season_Spring,tfa_season_Summer,tfa_season_Winter,dt_span,age_105,age_15,age_30,age_45,age_60,age_75,age_90,age_NA,age_Unphysical
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NA,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28,1,0,0,0,0,0,0,0,0,1,0,2009,3,19,0,0,0,1,0,0,0,0,1,0,0,70025,0,0,0,0,0,0,0,1,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,45,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,5,25,0,0,1,0,0,0,0,0,1,0,0,2009,5,23,0,0,0,0,0,1,0,0,1,0,0,22311,0,0,0,1,0,0,0,0,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,60,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,9,28,0,1,0,0,0,0,0,1,0,0,0,2009,6,9,0,1,0,0,0,0,0,0,0,1,0,2833,0,0,0,0,1,0,0,0,0
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,45,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,12,5,1,0,0,0,0,0,0,0,0,0,1,2009,10,31,0,0,0,0,0,1,0,1,0,0,0,64711,0,0,0,1,0,0,0,0,0
4,87mebub9p4,2010-09-14,20091208061105,-unknown-,45,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,9,14,0,1,0,0,0,0,0,1,0,0,0,2009,12,8,0,1,0,0,0,0,0,0,0,0,1,64135,0,0,0,1,0,0,0,0,0
5,osr2jwljor,2010-01-01,20100101215619,-unknown-,NA,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,2010,1,1,0,0,0,0,1,0,0,0,0,0,1,2010,1,1,0,0,0,0,1,0,0,0,0,0,1,7421,0,0,0,0,0,0,0,1,0
6,lsw9q7uk0j,2010-01-02,20100102012558,FEMALE,60,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,2010,1,2,0,0,0,0,0,1,0,0,0,0,1,2010,1,2,0,0,0,0,0,1,0,0,0,0,1,81242,0,0,0,0,1,0,0,0,0
7,0d01nltbrs,2010-01-03,20100103191905,FEMALE,60,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,2010,1,3,0,0,0,0,0,0,1,0,0,0,1,2010,1,3,0,0,0,0,0,0,1,0,0,0,1,16855,0,0,0,0,1,0,0,0,0
8,a1vcnhxeij,2010-01-04,20100104004211,FEMALE,60,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,2010,1,4,1,0,0,0,0,0,0,0,0,0,1,2010,1,4,1,0,0,0,0,0,0,0,0,0,1,83869,0,0,0,0,1,0,0,0,0
9,6uh8zyj2gn,2010-01-04,20100104023758,-unknown-,60,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,2010,1,4,1,0,0,0,0,0,0,0,0,0,1,2010,1,4,1,0,0,0,0,0,0,0,0,0,1,76922,0,0,0,0,1,0,0,0,0


In [36]:
feature_OHE = ['gender', 
               'signup_method', 
               'signup_flow', 
               'language', 
               'affiliate_channel', 
               'affiliate_provider', 
               'first_affiliate_tracked', 
               'signup_app', 
               'first_device_type', 
               'first_browser']

**Label Encoding:**<br/>
* Prerequisite for One-Hot-Encoding.
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html<br/>

In [37]:
from sklearn import preprocessing
LBL = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()
LE_map1=dict()
for cat_var in feature_OHE:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var].fillna('none'))
    LE_vars.append(LE_var)
    LE_map1[cat_var]=dict(zip(LBL.classes_, LBL.transform(LBL.classes_))) ## Here you generate the mapping dictionary
    LE_map[cat_var]=LBL.classes_
print ("Label-encoded feaures: %s" % (LE_vars))

Label Encoding gender
Label Encoding signup_method
Label Encoding signup_flow
Label Encoding language
Label Encoding affiliate_channel
Label Encoding affiliate_provider
Label Encoding first_affiliate_tracked
Label Encoding signup_app
Label Encoding first_device_type
Label Encoding first_browser
Label-encoded feaures: ['gender_le', 'signup_method_le', 'signup_flow_le', 'language_le', 'affiliate_channel_le', 'affiliate_provider_le', 'first_affiliate_tracked_le', 'signup_app_le', 'first_device_type_le', 'first_browser_le']


In [38]:
LE_map1

{'affiliate_channel': {'api': 0,
  'content': 1,
  'direct': 2,
  'other': 3,
  'remarketing': 4,
  'sem-brand': 5,
  'sem-non-brand': 6,
  'seo': 7},
 'affiliate_provider': {'baidu': 0,
  'bing': 1,
  'craigslist': 2,
  'daum': 3,
  'direct': 4,
  'email-marketing': 5,
  'facebook': 6,
  'facebook-open-graph': 7,
  'google': 8,
  'gsp': 9,
  'meetup': 10,
  'naver': 11,
  'other': 12,
  'padmapper': 13,
  'vast': 14,
  'wayn': 15,
  'yahoo': 16,
  'yandex': 17},
 'first_affiliate_tracked': {'linked': 0,
  'local ops': 1,
  'marketing': 2,
  'none': 3,
  'omg': 4,
  'product': 5,
  'tracked-other': 6,
  'untracked': 7},
 'first_browser': {'-unknown-': 0,
  'AOL Explorer': 1,
  'Android Browser': 2,
  'Apple Mail': 3,
  'Arora': 4,
  'Avant Browser': 5,
  'BlackBerry Browser': 6,
  'Camino': 7,
  'Chrome': 8,
  'Chrome Mobile': 9,
  'Chromium': 10,
  'CometBird': 11,
  'Comodo Dragon': 12,
  'Conkeror': 13,
  'CoolNovo': 14,
  'Crazy Browser': 15,
  'Epic': 16,
  'Firefox': 17,
  'Flock

**One Hot Encoding:**<br/>
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html<br/>

In [39]:
OHE = preprocessing.OneHotEncoder(sparse=False)
start=time.time()
OHE.fit(full_data[LE_vars])
OHE_data=OHE.transform(full_data[LE_vars])
                                   
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))


OHE_vars = [var[:-3] + '_' + str(level).replace(' ','_')\
                for var in feature_OHE for level in LE_map[var]]

print ("OHE size :" ,OHE_data.shape)
print ("One-hot encoded catgorical feature samples : %s" % (OHE_vars[:100]))

One-hot-encoding finished in 1.751148 seconds
OHE size : (275547, 154)
One-hot encoded catgorical feature samples : ['gen_-unknown-', 'gen_FEMALE', 'gen_MALE', 'gen_OTHER', 'signup_met_basic', 'signup_met_facebook', 'signup_met_google', 'signup_met_weibo', 'signup_f_0', 'signup_f_1', 'signup_f_2', 'signup_f_3', 'signup_f_4', 'signup_f_5', 'signup_f_6', 'signup_f_8', 'signup_f_10', 'signup_f_12', 'signup_f_14', 'signup_f_15', 'signup_f_16', 'signup_f_20', 'signup_f_21', 'signup_f_23', 'signup_f_24', 'signup_f_25', 'langu_-unknown-', 'langu_ca', 'langu_cs', 'langu_da', 'langu_de', 'langu_el', 'langu_en', 'langu_es', 'langu_fi', 'langu_fr', 'langu_hr', 'langu_hu', 'langu_id', 'langu_is', 'langu_it', 'langu_ja', 'langu_ko', 'langu_nl', 'langu_no', 'langu_pl', 'langu_pt', 'langu_ru', 'langu_sv', 'langu_th', 'langu_tr', 'langu_zh', 'affiliate_chan_api', 'affiliate_chan_content', 'affiliate_chan_direct', 'affiliate_chan_other', 'affiliate_chan_remarketing', 'affiliate_chan_sem-brand', 'affili

In [40]:
full_data = pd.concat((full_data, pd.DataFrame(OHE_data,columns=OHE_vars)), axis = 1)

In [41]:
full_data

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,dac_year,dac_month,dac_day,dac_wd_1,dac_wd_2,dac_wd_3,dac_wd_4,dac_wd_5,dac_wd_6,dac_wd_7,dac_season_Fall,dac_season_Spring,dac_season_Summer,dac_season_Winter,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,tfa_season_Fall,tfa_season_Spring,tfa_season_Summer,tfa_season_Winter,dt_span,age_105,age_15,age_30,age_45,age_60,age_75,age_90,age_NA,age_Unphysical,gender_le,signup_method_le,signup_flow_le,language_le,affiliate_channel_le,affiliate_provider_le,first_affiliate_tracked_le,signup_app_le,first_device_type_le,first_browser_le,gen_-unknown-,gen_FEMALE,gen_MALE,gen_OTHER,signup_met_basic,signup_met_facebook,signup_met_google,signup_met_weibo,signup_f_0,signup_f_1,signup_f_2,signup_f_3,signup_f_4,signup_f_5,signup_f_6,signup_f_8,signup_f_10,signup_f_12,signup_f_14,signup_f_15,signup_f_16,signup_f_20,signup_f_21,signup_f_23,signup_f_24,signup_f_25,langu_-unknown-,langu_ca,langu_cs,langu_da,langu_de,langu_el,langu_en,langu_es,langu_fi,langu_fr,langu_hr,langu_hu,langu_id,langu_is,langu_it,langu_ja,langu_ko,langu_nl,langu_no,langu_pl,langu_pt,langu_ru,langu_sv,langu_th,langu_tr,langu_zh,affiliate_chan_api,affiliate_chan_content,affiliate_chan_direct,affiliate_chan_other,affiliate_chan_remarketing,affiliate_chan_sem-brand,affiliate_chan_sem-non-brand,affiliate_chan_seo,affiliate_provi_baidu,affiliate_provi_bing,affiliate_provi_craigslist,affiliate_provi_daum,affiliate_provi_direct,affiliate_provi_email-marketing,affiliate_provi_facebook,affiliate_provi_facebook-open-graph,affiliate_provi_google,affiliate_provi_gsp,affiliate_provi_meetup,affiliate_provi_naver,affiliate_provi_other,affiliate_provi_padmapper,affiliate_provi_vast,affiliate_provi_wayn,affiliate_provi_yahoo,affiliate_provi_yandex,first_affiliate_trac_linked,first_affiliate_trac_local_ops,first_affiliate_trac_marketing,first_affiliate_trac_none,first_affiliate_trac_omg,first_affiliate_trac_product,first_affiliate_trac_tracked-other,first_affiliate_trac_untracked,signup__Android,signup__Moweb,signup__Web,signup__iOS,first_device_t_Android_Phone,first_device_t_Android_Tablet,first_device_t_Desktop_(Other),first_device_t_Mac_Desktop,first_device_t_Other/Unknown,first_device_t_SmartPhone_(Other),first_device_t_Windows_Desktop,first_device_t_iPad,first_device_t_iPhone,first_brow_-unknown-,first_brow_AOL_Explorer,first_brow_Android_Browser,first_brow_Apple_Mail,first_brow_Arora,first_brow_Avant_Browser,first_brow_BlackBerry_Browser,first_brow_Camino,first_brow_Chrome,first_brow_Chrome_Mobile,first_brow_Chromium,first_brow_CometBird,first_brow_Comodo_Dragon,first_brow_Conkeror,first_brow_CoolNovo,first_brow_Crazy_Browser,first_brow_Epic,first_brow_Firefox,first_brow_Flock,first_brow_Google_Earth,first_brow_Googlebot,first_brow_IBrowse,first_brow_IE,first_brow_IE_Mobile,first_brow_IceDragon,first_brow_IceWeasel,first_brow_Iron,first_brow_Kindle_Browser,first_brow_Maxthon,first_brow_Mobile_Firefox,first_brow_Mobile_Safari,first_brow_Mozilla,first_brow_NetNewsWire,first_brow_Nintendo_Browser,first_brow_OmniWeb,first_brow_Opera,first_brow_Opera_Mini,first_brow_Opera_Mobile,first_brow_Outlook_2007,first_brow_PS_Vita_browser,first_brow_Pale_Moon,first_brow_Palm_Pre_web_browser,first_brow_RockMelt,first_brow_Safari,first_brow_SeaMonkey,first_brow_Silk,first_brow_SiteKiosk,first_brow_SlimBrowser,first_brow_Sogou_Explorer,first_brow_Stainless,first_brow_TenFourFox,first_brow_TheWorld_Browser,first_brow_UC_Browser,first_brow_Yandex.Browser,first_brow_wOSBrowser
0,gxn3p5htnn,2010-06-28,20090319043255,-unknown-,NA,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28,1,0,0,0,0,0,0,0,0,1,0,2009,3,19,0,0,0,1,0,0,0,0,1,0,0,70025,0,0,0,0,0,0,0,1,0,0,1,0,6,2,4,7,2,3,8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

## Model Building

### Airbnb Evaluation: NDCG

https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings#evaluation

In [42]:
# From Kaggle Kernels

from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelBinarizer

def dcg_score(y_true, y_score, k=5):
    
    """
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    
    """
    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.
        
    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """
    lb = LabelBinarizer()
    lb.fit(range(len(predictions) + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)

In [43]:
labels_le = preprocessing.LabelEncoder()
ytrain = labels_le.fit_transform(labels)
label_map = dict(zip(labels_le.classes_, labels_le.transform(labels_le.classes_)))

In [44]:
label_map

{'AU': 0,
 'CA': 1,
 'DE': 2,
 'ES': 3,
 'FR': 4,
 'GB': 5,
 'IT': 6,
 'NDF': 7,
 'NL': 8,
 'PT': 9,
 'US': 10,
 'other': 11}

### Random Forest

Here you should use holdout validation (even though I didn't do that for simple).

In [45]:
xtrain = full_data.drop(['id','age','date_account_created','timestamp_first_active'],axis=1).drop(feature_OHE+LE_vars,axis=1)[:train_data.shape[0]]
xtest = full_data.drop(['id','age','date_account_created','timestamp_first_active'],axis=1).drop(feature_OHE+LE_vars,axis=1)[train_data.shape[0]:]

In [46]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [47]:
N_ESTIMATORS = 50
RANDOM_STATE = 2017
MAX_DEPTH = 9
RF = RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                 max_depth=MAX_DEPTH,
                                 random_state=RANDOM_STATE)

In [48]:
RF.fit(xtrain,ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=2017,
            verbose=0, warm_start=False)

In [49]:
predict = RF.predict_proba(xtrain)

In [50]:
np.argsort(predict[1])

array([ 9,  0,  8,  2,  1,  5,  6,  3,  4, 11, 10,  7])

In [52]:
predict[1]

array([ 0.00218461,  0.00826545,  0.00749203,  0.01532227,  0.02852751,
        0.01374148,  0.01392895,  0.46289443,  0.00482835,  0.00163559,
        0.39200267,  0.04917666])

In [54]:
k_ndcg = 5
train_ndcg_score = ndcg_score(ytrain[:1000], predict[:1000], k = k_ndcg)
train_ndcg_score

0.80757727346728769

In [64]:
id_test = test_data['id']

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += labels_le.inverse_transform(np.argsort(predict[i])[::-1])[:5].tolist()

#Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('sub.csv',index=False)